In [1]:
%pylab inline
import sys
sys.path.append('../build_gcc/')
import versor as vsr

Populating the interactive namespace from numpy and matplotlib


versor/cga.py:4: RuntimeWarning: to-Python converter for vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)1, (short)2, (short)4, (short)8, (short)16> > already registered; second conversion method ignored.
  from libversor import (Biv, Bst, Cir, Con, Dil, Dll, Dlp, Dls, Drb, Drt, Drv,
versor/cga.py:4: RuntimeWarning: to-Python converter for vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)9, (short)10, (short)12> > already registered; second conversion method ignored.
  from libversor import (Biv, Bst, Cir, Con, Dil, Dll, Dlp, Dls, Drb, Drt, Drv,


In [2]:
np.set_printoptions(precision=2, suppress=True)

# Hand Eye Calibration

In [3]:
M_object_in_world = vsr.Trs.from_vector([1,1,0.1]) * vsr.Rot.from_bivector(vsr.Biv(1,0,0) * np.pi/4.0)
print(M_object_in_world)

[ 0.92 -0.38  0.    0.   -0.65 -0.27 -0.05  0.02]


In [4]:
log = M_object_in_world.log()
log

array([-0.39,  0.  ,  0.  , -0.67, -0.28, -0.05])

In [5]:
M_eye_in_hand = vsr.Trs.from_vector([0.1,0.2,0.3]) * vsr.Rot.from_bivector(vsr.Biv(0,1,0) * np.pi/6.)
print(M_eye_in_hand)

[ 0.97  0.   -0.26  0.   -0.09 -0.1  -0.13 -0.03]


In [6]:
M_hand_in_world = vsr.Trs.from_vector([0.0,0.0,1.0]) * vsr.Rot.from_bivector(vsr.Biv(1,1,1).unit() * np.pi/6.)
M_hand_in_world

array([ 0.97, -0.15, -0.15, -0.15, -0.07, -0.07, -0.48,  0.07])

In [7]:
M_object_in_eye = M_eye_in_hand.rev() * M_hand_in_world.rev() * M_object_in_world
print(M_object_in_eye.to_array())

[ 0.87 -0.24  0.43  0.02 -0.34 -0.11  0.84 -0.19]


In [8]:
print(M_hand_in_world)

[ 0.97 -0.15 -0.15 -0.15 -0.07 -0.07 -0.48  0.07]


In [9]:
print(M_hand_in_world.log().mot())

[ 0.97 -0.15 -0.15 -0.15 -0.07 -0.07 -0.48  0.07]


In [10]:
pose_pairs = np.array([(M_hand_in_world,
               M_eye_in_hand.rev() * M_hand_in_world.rev() * M_object_in_world) 
              for M_hand_in_world in [vsr.Trs.from_vector(vsr.Vec(*np.random.rand(3))) *
                                      vsr.Rot.from_bivector(vsr.Biv(*np.random.rand(3)).unit() * 
                                                            np.random.rand() * np.pi)
                                      for i in range(4)]])

In [74]:
M_object_in_world = vsr.Trs.from_vector([1,1,0.1]) * vsr.Rot.from_bivector(vsr.Biv(1,0,0) * np.pi/4.0)
M_eye_in_hand = vsr.Trs.from_vector([-0.5,0.2,-0.3]) * vsr.Rot.from_bivector(vsr.Biv(1,1,1).unit() * np.pi/6.)
pose_pairs = np.array([(M_hand_in_world,
               (M_eye_in_hand.rev() * M_hand_in_world.rev() * M_object_in_world))
              for M_hand_in_world in [vsr.Trs.from_vector(vsr.Vec(*np.random.rand(3))) *
                                      vsr.Rot.from_bivector(vsr.Biv(*np.random.rand(3)).unit() * 
                                                            np.random.rand() * np.pi)
                                      for i in range(4)]])
n = len(pose_pairs)
As = pose_pairs[:,0]
Bs = pose_pairs[:,1]
LAs = []
LBs = []
MAs = []
MBs = []
for i in range(n):
    for j in range(i+1,n):
        MBs.append((Bs[j] * Bs[i].rev()))     
        MAs.append((As[j].rev() * As[i]))
        LBs.append((Bs[j] * Bs[i].rev()).log())     
        LAs.append((As[j].rev() * As[i]).log())
print(len(LAs))

6


In [75]:
print LBs[1].unit().spin(M_eye_in_hand)
print(LAs[1].unit())

[ 0.53 -0.18  0.83 -0.5  -0.09 -0.34]
[ 0.53 -0.18  0.83 -0.5  -0.09 -0.34]


In [76]:
[np.allclose(M_object_in_world.to_array(),
             (M_hand_in_world * M_eye_in_hand * M_object_in_eye).to_array())
             for (M_hand_in_world, M_object_in_eye) in pose_pairs]

[True, True, True, True]

In [77]:
[np.allclose(a[0],b[0]) for a,b in zip(MAs, MBs)]

[True, True, True, True, True, True]

In [78]:
def skew(v):
    skv = roll(roll(diag(v.flatten()), 1, 1), -1, 0)
    return skv - skv.T

In [79]:
skew(np.array([1,2,3]))

array([[ 0, -3,  2],
       [ 3,  0, -1],
       [-2,  1,  0]])

In [112]:
Ds = []
for MA, MB in zip(MAs, MBs):
    D = np.zeros((6,8))
#     a = MA.log().unit().to_array().copy()[:3]
#     b = MB.log().unit().to_array().copy()[:3]
#     aprime = MA.log().unit().to_array().copy()[3:]
#     bprime = MB.log().unit().to_array().copy()[3:]
#     print(MA)
    

#     MA = MA.spin(vsr.Trs.from_vector(vsr.Vec(*np.random.randn(3) * 0.001 )) *
#                                       vsr.Rot.from_bivector(vsr.Biv(*np.random.rand(3)).unit() * 
#                                                             np.random.randn() * 0.01))
#     MB = MB.spin(vsr.Trs.from_vector(vsr.Vec(*np.random.randn(3) * 0.001 )) *
#                                       vsr.Rot.from_bivector(vsr.Biv(*np.random.rand(3)).unit() * 
#                                                             np.random.randn() * 0.01))

#     print(MA)

    a = MA.to_array().copy()[1:4]
    a = np.array([a[2], -a[1], a[0]])
    b = MB.to_array().copy()[1:4]
    b = np.array([b[2], -b[1], b[0]])
    aprime = MA.to_array().copy()[4:7]
    bprime = MB.to_array().copy()[4:7]
    # Upper 3
    D[:3,0] = a - b
    D[:3,1:4] = skew(a + b)
    # Lower 3
    D[3:,0] = aprime - bprime
    D[3:,1:4] = skew(aprime + bprime)
    D[3:,4] = a - b 
    D[3:,5:8] = skew(a + b)
    Ds.append(D)
Ds = np.array(Ds).reshape(-1,8)

In [113]:
Ds = []
for LA, LB in zip(LAs, LBs):
    LA = LA.unit()
    LB = LB.unit()
    D = np.zeros((6,8))
    
    a = LA.to_array().copy()[:3]
    a = np.array([a[2], -a[1], a[0]])
    b = LB.to_array().copy()[:3]
    b = np.array([b[2], -b[1], b[0]])
    aprime = LA.to_array().copy()[3:]
    bprime = LB.to_array().copy()[3:]
    # Upper 3
    D[:3,0] = a - b
    D[:3,1:4] = skew(a + b)
    # Lower 3
    D[3:,0] = aprime - bprime
    D[3:,1:4] = skew(aprime + bprime)
    D[3:,4] = a - b 
    D[3:,5:8] = skew(a + b)  
    Ds.append(D)
Ds = np.array(Ds).reshape(-1,8)

In [114]:
[U, s, Vt] = np.linalg.svd(Ds)
print(Vt.T)
v7 = Vt.T[:,-2]
v8 = Vt.T[:,-1]
u1 = v7[:4]
v1 = v7[4:]
u2 = v8[:4]
v2 = v8[4:]

[[-0.06  0.15 -0.01  0.19 -0.17 -0.24  0.92  0.02]
 [-0.62 -0.74 -0.06 -0.22  0.05  0.03  0.14  0.  ]
 [-0.61  0.53  0.29 -0.33 -0.35 -0.1  -0.14 -0.  ]
 [ 0.21 -0.14  0.89 -0.21  0.29  0.01  0.14  0.  ]
 [ 0.02  0.04 -0.03 -0.09  0.    0.22  0.09 -0.96]
 [-0.13 -0.22  0.34  0.74 -0.41 -0.16 -0.23 -0.16]
 [ 0.29 -0.18  0.05 -0.24 -0.71  0.53  0.1   0.15]
 [ 0.31 -0.21 -0.07 -0.38 -0.3  -0.76 -0.11 -0.15]]


In [108]:
a = np.inner(u1,v1)
b = np.inner(u1,v2) + np.inner(u2,v1)
c = np.inner(u2,v2)
[s1, s2] = np.roots([a,b,c])
print(s1, s2)

(0.92171771269433955, -0.17738328934948891)


In [109]:
val1 = (s1**2 * np.inner(u1,u1)) + (2 * s1 * np.inner(u1,u2)) + (np.inner(u2,u2))
val2 = (s2**2 * np.inner(u1,u1)) + (2 * s2 * np.inner(u1,u2)) + (np.inner(u2,u2))

if val1 > val2:
    s = s1
    val = val1
else:
    s = s2
    val = val2

lambda2 = np.sqrt(1./val)
lambda1 = s * lambda2
print(lambda1)
print(lambda2)

0.891239859849
0.966933636595


In [110]:
m_arr = lambda1 * v7 + lambda2 * v8
print(M_eye_in_hand.to_array())
# m_arr[1:4] = np.array([m_arr[3], -m_arr[2], m_arr[1]])
# m_arr[4:8] = np.roll(m_arr[4:8], -1)
print(m_arr)

m = vsr.Mot(*m_arr)
print(m * m.rev())

[ 0.97 -0.15 -0.15 -0.15  0.25 -0.11  0.12 -0.07]
[ 0.97  0.15 -0.15  0.15 -0.13 -0.01 -0.    0.84]
[ 1.    0.    0.    0.    0.    0.    0.    1.67]


In [103]:
M_eye_in_hand * M_eye_in_hand.rev()

array([ 1.,  0.,  0.,  0.,  0., -0.,  0.,  0.])

In [104]:
def polar_decomposition(x):
    xx = x * x.rev()
    sinv = (vsr.Sca(1.0) + (vsr.Drt(- xx[7] / (2 * xx[0]))))
    return x * sinv

In [38]:
m2 = polar_decomposition(m)
print(m2)

[-0.87  0.   -0.5  -0.   -0.05  0.04 -0.09 -0.03]


In [39]:
M_eye_in_hand

array([ 0.87,  0.  , -0.5 ,  0.  ,  0.12, -0.09,  0.1 , -0.05])

In [26]:
(np.sqrt(1./val2) * s2) * v7 + np.sqrt(1./val2) * v8

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in add
  if __name__ == '__main__':


array([ nan, -inf,  nan,  nan, -inf,  nan,  inf,  nan])

In [27]:
(np.sqrt(1./val1) * s1) * v7 + np.sqrt(1./val1) * v8

array([-0.87, -0.  ,  0.5 ,  0.  , -0.05,  0.12, -0.09,  0.1 ])

In [28]:
M_eye_in_hand

array([ 0.87,  0.  , -0.5 ,  0.  ,  0.12, -0.09,  0.1 , -0.05])

In [29]:
np.inner(M_eye_in_hand.to_array()[:4], M_eye_in_hand.to_array()[:4])

1.0